In [6]:
import pandas as pd
import altair as alt
import numpy as np
import sqlite3

# Create DataBase and Add Table

In [7]:
conn = sqlite3.connect('Netflix.db')

cur = conn.cursor()

# cur.execute("""CREATE TABLE NFLX (
#     date string,
#     low float,
#     open float,
#     volume integer,
#     high float,
#     close float
# )""")

# Fill Table

In [8]:
path = 'C:/Users/Blake Dennett/Downloads/Spring2023/appliedProgramming/Data/stock_market_data/sp500/csv/j-p/nflx.csv'
df = pd.read_csv(path)
# iterate through each row of dataframe
for r in df.iterrows():
    row = r[1].tolist()
    to_insert = tuple(row[0:6])
    # add row into db
    # cur.execute(f"INSERT INTO NFLX VALUES {to_insert}")

conn.commit()

five_rows = pd.read_sql_query( 
    'SELECT * FROM NFLX LIMIT 5',
    conn)

five_rows

    

,date,low,open,volume,high,close
0,23-05-2002,1.145714,1.156429,104790000,1.242857,1.196429
1,24-05-2002,1.197143,1.214286,11104800,1.225000,1.210000
2,28-05-2002,1.157143,1.213571,6609400,1.232143,1.157143
3,29-05-2002,1.085714,1.164286,6757800,1.164286,1.103571
4,30-05-2002,1.071429,1.107857,10154200,1.107857,1.071429


# Deleting Duplicate Data

In [9]:
# cur.execute("DELETE FROM nflx WHERE date NOT IN (SELECT MIN(date) FROM nflx GROUP BY date);")

# Filtering Data

In [10]:
cur.execute("SELECT * FROM NFLX WHERE date = '23-05-2002'")
cur.fetchall()

[('23-05-2002',
  1.1457140445709229,
  1.1564290523529053,
  104790000,
  1.2428569793701172,
  1.1964290142059326)]

# Creating New Data Table

In [ ]:
# cur.execute("""CREATE TABLE NFLX_ANALYSIS (
#             date string,
#             difference float
# )""")

# Fill Date Column From One Table to Another

In [77]:
data_transfer = """INSERT INTO NFLX_ANALYSIS (date) 
SELECT date FROM NFLX"""
# cur.execute(data_transfer)

data_display = """SELECT * 
FROM NFLX_ANALYSIS 
LIMIT 5"""

print(pd.read_sql_query(data_display, conn))

         date difference
0  23-05-2002       None
1  24-05-2002       None
2  28-05-2002       None
3  29-05-2002       None
4  30-05-2002       None


# Use Join to add difference column

In [85]:
# make a difference column
query = """ALTER TABLE NFLX
ADD COLUMN difference FLOAT;"""
# cur.execute(query)

# subtract the high from the low
query = """UPDATE NFLX
SET difference = high - low"""
# cur.execute(query)

# enter difference into NFLX_ANALYSIS
diff_transfer = """INSERT INTO NFLX_ANALYSIS (difference, date)
SELECT difference, date FROM NFLX"""
# cur.execute(diff_transfer)

# JOINING HERE
query = """INSERT INTO NFLX_ANALYSIS (difference)
SELECT NFLX.difference
FROM NFLX
INNER JOIN NFLX_ANALYSIS ON NFLX.date = NFLX_ANALYSIS.date"""
# cur.execute(query)


# cur.execute("""DELETE FROM NFLX_ANALYSIS""")


print(pd.read_sql_query("""SELECT * FROM NFLX_ANALYSIS LIMIT 5""", conn))
print(pd.read_sql_query("""SELECT * FROM NFLX_ANALYSIS WHERE difference IS NOT NULL""", conn))
# print(pd.read_sql_query("""SELECT * FROM NFLX_ANALYSIS WHERE date = '23-05-2002'""", conn))


         date difference
0  23-05-2002       None
1  24-05-2002       None
2  28-05-2002       None
3  29-05-2002       None
4  30-05-2002       None
      date  difference
0     None    0.097143
1     None    0.027857
2     None    0.075000
3     None    0.078572
4     None    0.036428
...    ...         ...
5171  None   10.089996
5172  None    7.509979
5173  None    7.570007
5174  None    9.480011
5175  None   12.910004

[5176 rows x 2 columns]
